<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/tpu_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!gdown --id 1-8nsWLseynVj6Z9-E12w1ywnffnqJftm
!gdown --id 1UszbNYQnlNrAcPQkBwvb1wKX21oRPiqb

Downloading...
From: https://drive.google.com/uc?id=1-8nsWLseynVj6Z9-E12w1ywnffnqJftm
To: /content/Uembeds306Epochs.npy
2.59GB [00:23, 111MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1UszbNYQnlNrAcPQkBwvb1wKX21oRPiqb
To: /content/Vembeds306Epochs.npy
2.59GB [00:31, 81.3MB/s]


In [79]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from time import time
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-rc1


In [14]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.75.65.194:8470']


INFO:tensorflow:Initializing the TPU system: 10.75.65.194:8470


INFO:tensorflow:Initializing the TPU system: 10.75.65.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [29]:
workers = ['/job:worker/replica:0/task:0/device:TPU:'+str(i) for i in range(8)]
workers

['/job:worker/replica:0/task:0/device:TPU:0',
 '/job:worker/replica:0/task:0/device:TPU:1',
 '/job:worker/replica:0/task:0/device:TPU:2',
 '/job:worker/replica:0/task:0/device:TPU:3',
 '/job:worker/replica:0/task:0/device:TPU:4',
 '/job:worker/replica:0/task:0/device:TPU:5',
 '/job:worker/replica:0/task:0/device:TPU:6',
 '/job:worker/replica:0/task:0/device:TPU:7']

In [0]:
class Index:
    def __init__(self, u, v, worker):
        self.embeddings = tf.math.l2_normalize(u, axis=0) + tf.math.l2_normalize(v, axis=0)
        self.squared_norms_embeddings = tf.expand_dims(tf.square(tf.norm(self.embeddings, axis=1)), axis=0)
        self.worker = worker

    def search(self, query_vector, top_k=None):
      with tf.device(worker):
        squared_norms_query_vector = tf.expand_dims(tf.square(tf.norm(query_vector, axis=1)), axis=0)
        dot_product = tf.reduce_sum(self.embeddings * query_vector, axis=1)
        distances = tf.maximum(self.squared_norms_embeddings + squared_norms_query_vector - 2 * dot_product, 0)
        sorted_indices =  tf.argsort(distances)
        if top_k:
            sorted_indices = sorted_indices[..., :top_k]
        nearest_distances = tf.reshape(tf.gather(distances[0], sorted_indices), shape=[-1, 1])
        return nearest_distances[..., 0], sorted_indices[0]

In [16]:
u_embeddings = np.load('Uembeds306Epochs.npy')
v_embeddings = np.load('Vembeds306Epochs.npy')
u_embeddings.shape, v_embeddings.shape

((1262996, 512), (1262996, 512))

In [0]:
u_embeddings = np.split(u_embeddings[:-4], 8, axis=0)
v_embeddings = np.split(v_embeddings[:-4], 8, axis=0)

In [64]:
## Place 1/8 of total embeddings on each TPU core
indices = []
for i, worker in enumerate(workers):
  with tf.device(worker):
    print('Building index on {}'.format(worker))
    indices.append(Index(u_embeddings[i], u_embeddings[i], worker))

Building index on /job:worker/replica:0/task:0/device:TPU:0
Building index on /job:worker/replica:0/task:0/device:TPU:1
Building index on /job:worker/replica:0/task:0/device:TPU:2
Building index on /job:worker/replica:0/task:0/device:TPU:3
Building index on /job:worker/replica:0/task:0/device:TPU:4
Building index on /job:worker/replica:0/task:0/device:TPU:5
Building index on /job:worker/replica:0/task:0/device:TPU:6
Building index on /job:worker/replica:0/task:0/device:TPU:7


In [0]:
def search(xq):
  D, I = [], []

  for i in range(8):
    d, idx = indices[i].search(xq, 1)
    D.append(d.numpy()[0])
    I.append(i*vecs_per_index + idx.numpy()[0])
    
  id_sorted = np.argsort(D)
  D = np.array(D)[id_sorted]
  I = np.array(I)[id_sorted]
  return D, I

In [97]:
vecs_per_index = 157874
n = 123 

split = 7 # [0, 7] . # Pick nth v
ector from given split

actual_n = vecs_per_index*split + n
xq = tf.nn.l2_normalize(u_embeddings[split][n]) + tf.nn.l2_normalize(v_embeddings[split][n])
xq = tf.reshape(xq, [1, -1])

s = time()
D, I = search(xq)
e = time()

print('Actual ID  :', actual_n)
print('Result IDs  :', I)
print('Distances   :', np.round(D, 4))
print('Time taken  :', e-s, 'secs')

Actual ID  : 1105241
Result IDs  : [1105241  438973  979042  633647   50252  945699  498614  175889]
Distances   : [1.9103 1.9247 1.9249 1.925  1.9252 1.9252 1.9256 1.9256]
Time taken  : 0.3391108512878418 secs
